# Enhanced Manhattan Real Estate Search

## Acquisition and processing of New York neighborhood venue data

### First Load necessary libraries

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  pyth

###  Get New York neighborhood data

In [14]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


### Load and explore the data

In [15]:
# load the data we just got
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

    
# the features key has the neigborhoods
neighborhoods_data = newyork_data['features']

# transform the data into a panda dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# loop through the data to populate the dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
# expect 5 boroughs and 306 neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
# look at the first entry
neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### For this project we are only looking at Manhattan data

In [16]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
print('The Manhattan dataframe has {} neighborhoods.'.format(
        manhattan_data.shape[0]
    )
)
manhattan_data

The Manhattan dataframe has 40 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


### Get the geographical coordinates of Manhattan

In [17]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


### Map the Manhattan data

In [18]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Set Client ID and Secret.  Hide for publish to Github

In [19]:
# The code was removed by Watson Studio for sharing.

### Get venue information from Foursquare for the neighborhoods

In [20]:
# define the function to do work
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# call it for all the neighborhoods
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
print(manhattan_venues.shape)
manhattan_venues.head()

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
(3263, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


In [21]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3263, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


### Analyze each neighborhood

In [22]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Sho

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Sho

### Print each neighborhood with the five most common venues

In [24]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0            Park  0.09
1  Clothing Store  0.06
2     Coffee Shop  0.05
3           Hotel  0.05
4   Women's Store  0.03


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.08
1                  Café  0.06
2           Pizza Place  0.04
3  Gym / Fitness Center  0.03
4        Cosmetics Shop  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1   American Restaurant  0.05
2                   Bar  0.05
3    Seafood Restaurant  0.05
4  Gym / Fitness Center  0.05


----Chelsea----
                 venue  freq
0               Bakery  0.05
1          Coffee Shop  0.05
2          Art Gallery  0.04
3  American Restaurant  0.04
4                Hotel  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1               Bakery  0.07
2         Cocktail Bar  0.05
3  American Restaurant  0.04
4         Dessert Shop  0.03


----Civic Center----
            

### Put that into a pandas dataframe

In [25]:
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Clothing Store,Coffee Shop,Hotel,Women's Store,Playground,Gym,Memorial Site,Sandwich Place,Shopping Mall
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Gym / Fitness Center,Cosmetics Shop,Gym,Wine Shop,Bookstore,Bar,Japanese Restaurant
2,Central Harlem,African Restaurant,American Restaurant,Bar,Seafood Restaurant,Gym / Fitness Center,French Restaurant,Chinese Restaurant,Library,Spa,Music Venue
3,Chelsea,Bakery,Coffee Shop,Art Gallery,American Restaurant,Hotel,Seafood Restaurant,French Restaurant,Ice Cream Shop,Wine Shop,Beer Bar
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Dessert Shop,Hotpot Restaurant,Salon / Barbershop,Optical Shop,Noodle House,Dim Sum Restaurant


## Acquisition and processing of available property data

### Data file downloaded from NY government sales data.  Cell hidden to protect credentials.

In [26]:
# The code was removed by Watson Studio for sharing.

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,372,10,NaN,C7,274 EAST 3RD,NaN,10009,9,1,10,"2,021","6,445",1900,2,C7,0,12/27/2020
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,374,1,NaN,C7,301-303 EAST 4TH STREET,NaN,10009,8,2,10,"1,501","6,929",1900,2,C7,"3,672,530",11/24/2020
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,376,54,NaN,C4,"719 EAST SIXTH STREET, 5C",NaN,10009,20,0,20,"2,302","9,750",1900,2,C4,"154,196",6/11/2021
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,386,53,NaN,C1,233 EAST 3 STREET,NaN,10009,29,0,29,"3,927","17,754",1910,2,C1,"7,500,000",6/14/2021
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,390,57,NaN,C7,197 EAST 7TH STREET,NaN,10009,10,1,11,"1,600","5,320",1910,2,C7,"3,425,000",2/11/2021


In [27]:
# remove columns we don't need, weren't populated correctly, or wouldn't be in a typical property search
df_data_2.drop(columns=['BOROUGH', 'BLOCK', 'LOT', 'EASEMENT', 'BUILDING CLASS AT PRESENT', 'APARTMENT NUMBER', 'RESIDENTIAL UNITS', 
                        'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET',
                        'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE', 'SALE DATE'], inplace=True)
df_data_2.head()

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,ADDRESS,ZIP CODE,YEAR BUILT,SALE PRICE
0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,274 EAST 3RD,10009,1900,0
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,301-303 EAST 4TH STREET,10009,1900,"3,672,530"
2,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,"719 EAST SIXTH STREET, 5C",10009,1900,"154,196"
3,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,233 EAST 3 STREET,10009,1910,"7,500,000"
4,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,197 EAST 7TH STREET,10009,1910,"3,425,000"


In [35]:
# Test geocode
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("197 E 7th St Manhattan")

print("197 E 7th St Manhattan: Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

197 E 7th St Manhattan: Latitude = 40.7248716, Longitude = -73.98021797713645


### Add Latitude and Longitude based on Address

In [36]:
# do a deep copy
manhattan_property_search_results = df_data_2.copy()
# get rid of apartment number information.  not needed for this and causes problem with geocode
manhattan_property_search_results['ADDRESS'] = manhattan_property_search_results['ADDRESS'].str.replace(',.*$', '', regex=True)
# Add NYC to address for lat/lon lookup
manhattan_property_search_results.ADDRESS = manhattan_property_search_results.ADDRESS + ' Manhattan'

from geopy.extra.rate_limiter import RateLimiter

locator = Nominatim(user_agent="myGeocoder")

# 1 - convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
manhattan_property_search_results['location'] = manhattan_property_search_results['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple).  if address fails, say None
manhattan_property_search_results['point'] = manhattan_property_search_results['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# Delete rows where address didn't return a location.  Normally, you would want to handle this, but for this prototype, just delete those rows
print ("Shape before filtering: ", manhattan_property_search_results.shape)
manhattan_property_search_filtered = manhattan_property_search_results.copy()
# now that the lookup is done, restore the original address column which included apartment number
manhattan_property_search_filtered['ADDRESS'] = df_data_2['ADDRESS']

manhattan_property_search_filtered = manhattan_property_search_filtered[manhattan_property_search_filtered['location'].notnull()]
print ("Shape after filtering out addresses where geocode didn't return a location: ", manhattan_property_search_filtered.shape)

# 4 - split point column into latitude, longitude and altitude columns
manhattan_property_search_filtered[['latitude', 'longitude', 'altitude']] = pd.DataFrame(manhattan_property_search_filtered['point'].tolist(), index=manhattan_property_search_filtered.index)
manhattan_property_search_filtered.head()

Shape before filtering:  (435, 9)
Shape after filtering out addresses where geocode didn't return a location:  (318, 9)


,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,ADDRESS,ZIP CODE,YEAR BUILT,SALE PRICE,location,point,latitude,longitude,altitude
0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,274 EAST 3RD,10009,1900,0,"(274, North 3rd Street, Manhattan, Riley Count...","(39.18019266383331, -96.5600831015777, 0.0)",39.180193,-96.560083,0.0
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,301-303 EAST 4TH STREET,10009,1900,"3,672,530","(301, East 4th Street, Manhattan Community Boa...","(40.722295, -73.979974, 0.0)",40.722295,-73.979974,0.0
2,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,"719 EAST SIXTH STREET, 5C",10009,1900,"154,196","(Sixth Street, Manhattan, Nye County, Nevada, ...","(38.5401105, -117.075693, 0.0)",38.540110,-117.075693,0.0
3,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,233 EAST 3 STREET,10009,1910,"7,500,000","(233, Alexander Street, Pearl-Meigs-Monroe, Ro...","(43.1506118, -77.5971389, 0.0)",43.150612,-77.597139,0.0
4,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,197 EAST 7TH STREET,10009,1910,"3,425,000","(197, East 7th Street, Alphabet City, Manhatta...","(40.7248716, -73.98021797713645, 0.0)",40.724872,-73.980218,0.0


In [37]:
# No longer need location, point, and altitude
manhattan_property_search_filtered.drop(columns=['location', 'point', 'altitude'], inplace=True)
manhattan_property_search_filtered.head()

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,ADDRESS,ZIP CODE,YEAR BUILT,SALE PRICE,latitude,longitude
0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,274 EAST 3RD,10009,1900,0,39.180193,-96.560083
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,301-303 EAST 4TH STREET,10009,1900,"3,672,530",40.722295,-73.979974
2,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,"719 EAST SIXTH STREET, 5C",10009,1900,"154,196",38.540110,-117.075693
3,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,233 EAST 3 STREET,10009,1910,"7,500,000",43.150612,-77.597139
4,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,197 EAST 7TH STREET,10009,1910,"3,425,000",40.724872,-73.980218


### Examine how the neighborhood names match between our property data and Foursquare data

In [38]:
manhattan_property_search_filtered['NEIGHBORHOOD'].unique()

array(['ALPHABET CITY', 'CHELSEA', 'CHINATOWN', 'CLINTON', 'EAST VILLAGE',
       'FASHION', 'GRAMERCY', 'GREENWICH VILLAGE-CENTRAL',
       'GREENWICH VILLAGE-WEST', 'HARLEM-CENTRAL', 'HARLEM-EAST',
       'HARLEM-UPPER', 'HARLEM-WEST', 'INWOOD', 'JAVITS CENTER',
       'KIPS BAY', 'LITTLE ITALY', 'LOWER EAST SIDE', 'MANHATTAN VALLEY',
       'MIDTOWN EAST', 'MIDTOWN WEST', 'MORNINGSIDE HEIGHTS',
       'MURRAY HILL', 'SOHO', 'TRIBECA', 'UPPER EAST SIDE (59-79)',
       'UPPER EAST SIDE (79-96)', 'UPPER EAST SIDE (96-110)',
       'UPPER WEST SIDE (59-79)', 'UPPER WEST SIDE (79-96)',
       'UPPER WEST SIDE (96-116)', 'WASHINGTON HEIGHTS LOWER',
       'WASHINGTON HEIGHTS UPPER'], dtype=object)

In [39]:
manhattan_data['Neighborhood'].unique()

array(['Marble Hill', 'Chinatown', 'Washington Heights', 'Inwood',
       'Hamilton Heights', 'Manhattanville', 'Central Harlem',
       'East Harlem', 'Upper East Side', 'Yorkville', 'Lenox Hill',
       'Roosevelt Island', 'Upper West Side', 'Lincoln Square', 'Clinton',
       'Midtown', 'Murray Hill', 'Chelsea', 'Greenwich Village',
       'East Village', 'Lower East Side', 'Tribeca', 'Little Italy',
       'Soho', 'West Village', 'Manhattan Valley', 'Morningside Heights',
       'Gramercy', 'Battery Park City', 'Financial District',
       'Carnegie Hill', 'Noho', 'Civic Center', 'Midtown South',
       'Sutton Place', 'Turtle Bay', 'Tudor City', 'Stuyvesant Town',
       'Flatiron', 'Hudson Yards'], dtype=object)

### Routines for finding closest neighborhood to each property address

In [40]:
# Functions curtesy of Analytics Vidhya
# Distance function
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Find Nearest Neighborhood function
def find_nearest(lat, long):
    distances = manhattan_data.apply(
        lambda row: dist(lat, long, row['Latitude'], row['Longitude']), 
        axis=1)
    return manhattan_data.loc[distances.idxmin(), 'Neighborhood']

### Use above functions to create new column of 'Foursquare Neighborhood' for the property data

In [41]:
manhattan_property_search_filtered['Foursquare Neighborhood'] = manhattan_property_search_filtered.apply(
    lambda row: find_nearest(row['latitude'], row['longitude']), 
    axis=1)

In [42]:
# To check the data frame if it has a new column of 'Foursquare Neighborhood' (for each and every property in the list)
manhattan_property_search_filtered[['NEIGHBORHOOD', 'Foursquare Neighborhood', 'latitude', 'longitude']].head(20)

,NEIGHBORHOOD,Foursquare Neighborhood,latitude,longitude
0,ALPHABET CITY,Battery Park City,39.180193,-96.560083
1,ALPHABET CITY,Lower East Side,40.722295,-73.979974
2,ALPHABET CITY,Battery Park City,38.540110,-117.075693
3,ALPHABET CITY,Inwood,43.150612,-77.597139
4,ALPHABET CITY,East Village,40.724872,-73.980218
5,ALPHABET CITY,East Village,40.726519,-73.979912
6,ALPHABET CITY,East Village,40.726174,-73.977616
7,ALPHABET CITY,East Village,40.726174,-73.977616
8,ALPHABET CITY,East Village,40.728449,-73.978546
9,ALPHABET CITY,East Village,40.725098,-73.983497


### Examining the differences in neighborhoods, found that a number of property addresses were not in Manhattan.  Would have to make a decision on how to handle this.  For the purpose of this prototype, just go leave as closest neighborhood

## Utilize the venue and property data to provide a client with properties meeting their criteria

### Assume the client wants a neighboorhood with a Grocery Store, Pharmacy, and Coffee Shop

In [43]:
manhattan_grouped.head()

,Neighborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Sho

In [44]:
# use the previous df that had a frequency score for every venue in every neighborhood
manhattan_neighborhood_scored = manhattan_grouped[['Neighborhood', 'Coffee Shop', 'Grocery Store', 'Pharmacy']].copy()
# add a total score for each neighborhood
manhattan_neighborhood_scored['Score'] = manhattan_neighborhood_scored.sum(numeric_only=True, axis=1)
#manhattan_neighborhood_scored['All Present'] = np.where((manhattan_neighborhood_scored['Coffee Shop'] > 0) & (manhattan_neighborhood_scored['Grocery Store'] > 0) & (manhattan_neighborhood_scored['Grocery Store'] > 0), True, False)
manhattan_neighborhood_scored['All Present'] = np.where((manhattan_neighborhood_scored['Coffee Shop'] > 0) & (manhattan_neighborhood_scored['Grocery Store'] > 0) & (manhattan_neighborhood_scored['Pharmacy'] > 0), True, False)
manhattan_neighborhood_scored.head()

,Neighborhood,Coffee Shop,Grocery Store,Pharmacy,Score,All Present
0,Battery Park City,0.053763,0.010753,0.0,0.064516,False
1,Carnegie Hill,0.077778,0.022222,0.0,0.100000,False
2,Central Harlem,0.000000,0.022727,0.0,0.022727,False
3,Chelsea,0.050000,0.010000,0.0,0.060000,False
4,Chinatown,0.020000,0.000000,0.0,0.020000,False


### Now that we have scored the neighborhoods, we can filter the search results.  We will require that all requested venues are present, then sort based on the score.

In [45]:
# examine which neighborhoods have all venues present
manhattan_neighborhoods_filtered = manhattan_neighborhood_scored.loc[manhattan_neighborhood_scored['All Present']]
print(manhattan_neighborhoods_filtered)
manhattan_neighborhoods_set = set (manhattan_neighborhoods_filtered['Neighborhood'])
manhattan_neighborhoods_set

           Neighborhood  Coffee Shop  Grocery Store  Pharmacy     Score  \
10             Flatiron     0.030000       0.010000  0.010000  0.050000   
11             Gramercy     0.041237       0.020619  0.010309  0.072165   
15               Inwood     0.019231       0.019231  0.019231  0.057692   
17       Lincoln Square     0.030303       0.010101  0.010101  0.050505   
19      Lower East Side     0.019231       0.019231  0.038462  0.076923   
25  Morningside Heights     0.071429       0.023810  0.023810  0.119048   
26          Murray Hill     0.040000       0.020000  0.010000  0.070000   
34           Turtle Bay     0.050000       0.010000  0.010000  0.070000   
37   Washington Heights     0.022472       0.022472  0.011236  0.056180   

    All Present  
10         True  
11         True  
15         True  
17         True  
19         True  
25         True  
26         True  
34         True  
37         True  


{'Flatiron',
 'Gramercy',
 'Inwood',
 'Lincoln Square',
 'Lower East Side',
 'Morningside Heights',
 'Murray Hill',
 'Turtle Bay',
 'Washington Heights'}

### Filter out property data rows that don't have all required venues present

In [46]:
manhattan_property_search_venues_present = manhattan_property_search_filtered.loc[manhattan_property_search_filtered['Foursquare Neighborhood'].isin(manhattan_neighborhoods_set)]
manhattan_property_search_venues_present.head()

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,ADDRESS,ZIP CODE,YEAR BUILT,SALE PRICE,latitude,longitude,Foursquare Neighborhood
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,301-303 EAST 4TH STREET,10009,1900,"3,672,530",40.722295,-73.979974,Lower East Side
3,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,233 EAST 3 STREET,10009,1910,"7,500,000",43.150612,-77.597139,Inwood
36,CLINTON,07 RENTALS - WALKUP APARTMENTS,2,446 WEST 55TH STREET,10019,1901,"4,550,000",40.767257,-73.988725,Lincoln Square
70,FASHION,07 RENTALS - WALKUP APARTMENTS,2,532 9 AVENUE,10018,1901,"4,000,000",40.808930,-73.953850,Morningside Heights
71,GRAMERCY,07 RENTALS - WALKUP APARTMENTS,2A,129 EAST 17TH STREET,10003,1900,0,40.735814,-73.986666,Gramercy


In [47]:
manhattan_property_search_venues_present.shape

(54, 10)

### Sort based on the score

In [48]:
# Add venue information to property data

#manhattan_property_search_venues_present.merge(manhattan_neighborhoods_filtered, left_on='Foursquare Neighborhood', right_on='Neighborhood')
manhattan_property_search_venues_present_with_scores = manhattan_property_search_venues_present.merge(manhattan_neighborhoods_filtered, left_on='Foursquare Neighborhood', right_on='Neighborhood')
manhattan_property_search_venues_present_with_scores.sort_values(['Score'], ascending=False, inplace=True)

In [49]:
manhattan_property_search_venues_present_with_scores

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,ADDRESS,ZIP CODE,YEAR BUILT,SALE PRICE,latitude,longitude,Foursquare Neighborhood,Neighborhood,Coffee Shop,Grocery Store,Pharmacy,Score,All Present
27,MANHATTAN VALLEY,07 RENTALS - WALKUP APARTMENTS,2B,353 WEST 115 STREET,10026,1900,"11,000,000",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
18,FASHION,07 RENTALS - WALKUP APARTMENTS,2,532 9 AVENUE,10018,1901,"4,000,000",40.808930,-73.953850,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
26,MANHATTAN VALLEY,07 RENTALS - WALKUP APARTMENTS,2B,344 MANHATTAN AVENUE,10026,1900,"1,623,468",40.803910,-73.957328,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
25,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2,204 WEST 121ST STREET,10027,1910,4,40.806486,-73.950625,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
24,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2A,213 WEST 115 STREET,10026,1900,"810,630",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
23,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2A,221 WEST 115 STREET,10026,1900,"1,547,968",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
22,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2,229 WEST 115 STREET,10026,1900,"8,867,518",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
21,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2,233 WEST 115 STREET,10026,1900,"3,294,478",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
20,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2B,241 WEST 115 STREET,10026,1900,"2,931,231",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True
19,HARLEM-CENTRAL,07 RENTALS - WALKUP APARTMENTS,2B,243 WEST 115 STREET,10026,1900,"4,799,045",40.805649,-73.960339,Morningside Heights,Morningside Heights,0.071429,0.023810,0.023810,0.119048,True


### Map the results

In [50]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))# create map of Manhattan using latitude and longitude values

map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label, score in zip(manhattan_property_search_venues_present_with_scores['latitude'], 
                                  manhattan_property_search_venues_present_with_scores['longitude'], 
                                  manhattan_property_search_venues_present_with_scores['ADDRESS'],
                                  manhattan_property_search_venues_present_with_scores['Score']):
    label = folium.Popup(label + ', Venue Score: ' + "{:.3f}".format(score), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.
